# SafeX — TFLite Triage Training (Binary + Multi‑class, EN+MS+ZH)

This notebook:
1) Generates a SAFE synthetic dataset (no real URLs/brands),
2) Trains a multi‑class text classifier (BENIGN + scam categories),
3) Trains an optional binary classifier (SCAM vs BENIGN),
4) Exports .tflite models you can drop into the Android app.

⚠️ Safety note: the generated texts are non-actionable templates with placeholders like `[LINK]` and do not include real domains or step-by-step instructions.


In [ ]:
# === 0) Install dependencies (matches Google's Model Maker notebook pattern) ===
!sudo apt -y install libportaudio2
!pip -q install tflite-model-maker
!pip -q uninstall -y tflite_support_nightly
!pip -q install tflite_support_nightly


In [ ]:
# === 1) Imports ===
import os, random, json
import pandas as pd
import numpy as np
import tensorflow as tf

from tflite_model_maker import model_spec, text_classifier
from tflite_model_maker.text_classifier import DataLoader

tf.get_logger().setLevel('ERROR')
random.seed(42)
np.random.seed(42)


## 2) Labels
We use 7 classes:
- `BENIGN`
- `INVESTMENT`, `ROMANCE`, `DELIVERY`, `JOB`, `IMPERSONATION`, `OTHER`

Binary label is derived:
- `BENIGN` → `BENIGN`
- anything else → `SCAM`


In [ ]:
# === 2) Labels ===
MULTICLASS_LABELS = [
    "BENIGN",
    "INVESTMENT",
    "ROMANCE",
    "DELIVERY",
    "JOB",
    "IMPERSONATION",
    "OTHER",
]

def to_binary_label(multiclass_label: str) -> str:
    return "BENIGN" if multiclass_label == "BENIGN" else "SCAM"


## 3) Safe template library (EN/MS/ZH)
These are non-actionable templates. They contain placeholders like:
- `[LINK]`, `[QR]`, `[AMOUNT]`, `[TIME]`, `[ORG]`, `[NAME]`

You can add more templates, but keep them non-actionable (no real domains, no step-by-step instructions).


In [ ]:
# === 3) Template library ===

TEMPLATES = {
  "en": {
    "BENIGN": [
      "Hi [NAME], are we still on for dinner at [TIME]?",
      "Please review the meeting agenda in the doc: [LINK]",
      "Your receipt for RM[AMOUNT] is ready. Thank you.",
      "Can you call me when you're free? No rush.",
      "Reminder: your appointment is at [TIME].",
    ],
    "INVESTMENT": [
      "Limited-time offer: easy returns with low effort. Learn more: [LINK]",
      "Guaranteed profit claims are common in scams. Check details before acting.",
      "Earn RM[AMOUNT] per day with a simple method. Start now: [LINK]",
      "Fast payout, no risk, instant approval. This is a common scam pattern.",
      "Exclusive group invitation for ‘quick earnings’. Verify first.",
    ],
    "ROMANCE": [
      "I feel a strong connection, but I need help with a small fee (red flag).",
      "Please keep our relationship secret and send support urgently (red flag).",
      "I can't video call now but trust me. Can you help with RM[AMOUNT]?",
      "I’m stuck overseas and need emergency help today (common romance scam).",
      "Let’s move to another app quickly (common manipulation pattern).",
    ],
    "DELIVERY": [
      "Your parcel has an issue. Resolve now to avoid return: [LINK]",
      "Delivery fee required today. Verify using official channels (red flag).",
      "Package status: action required within [TIME]. Details: [LINK]",
      "We couldn't deliver your item. Confirm address here: [LINK]",
      "Small fee requested to release parcel. Common scam signal.",
    ],
    "JOB": [
      "Work from home, high pay, no experience. Join now: [LINK]",
      "Immediate hiring, guaranteed income. Provide details now (red flag).",
      "Earn commissions by completing simple tasks. Start: [LINK]",
      "Pay a ‘registration fee’ to secure the job (common scam).",
      "We need your documents urgently to proceed (pressure tactic).",
    ],
    "IMPERSONATION": [
      "This is [ORG] support. Verify your account now: [LINK]",
      "Official-looking message asking for urgent action is a red flag.",
      "Hello, I'm your boss. Need a quick purchase urgently (common scam).",
      "Your account will be suspended unless you act now (fear tactic).",
      "Authority claim + urgency + money request is high-risk.",
    ],
    "OTHER": [
      "You won a prize. Claim now by paying a small fee (common scam).",
      "Urgent verification required. Do not share codes or passwords.",
      "Clicking unknown links can be dangerous. Use SafeX Scan: [LINK]",
      "Your device is infected. Download this now (fake warning pattern).",
      "Threatening language demanding immediate payment is suspicious.",
    ],
  },

  "ms": {
    "BENIGN": [
      "Hai [NAME], kita jumpa pukul [TIME] ya?",
      "Boleh semak dokumen ini bila sempat: [LINK]",
      "Resit untuk RM[AMOUNT] sudah tersedia. Terima kasih.",
      "Boleh telefon saya bila awak lapang? Tak urgent.",
      "Peringatan: temujanji anda pada [TIME].",
    ],
    "INVESTMENT": [
      "Pulangan mudah dan cepat, risiko rendah. Ketahui lebih lanjut: [LINK]",
      "Janji 'untung dijamin' sering tanda penipuan. Sila semak dulu.",
      "Jana RM[AMOUNT] sehari dengan cara mudah. Mula sekarang: [LINK]",
      "Bayaran cepat, tanpa risiko, kelulusan segera (corak penipuan biasa).",
      "Jemputan kumpulan 'duit cepat'. Sahkan sebelum bertindak.",
    ],
    "ROMANCE": [
      "Saya sayang awak, tapi saya perlukan bantuan segera RM[AMOUNT] (amaran).",
      "Tolong rahsiakan hubungan kita dan bantu segera (taktik manipulasi).",
      "Saya tak boleh video call sekarang, percayalah. Boleh bantu RM[AMOUNT]?",
      "Saya tersekat di luar negara dan perlukan bantuan hari ini (amaran).",
      "Jom pindah ke aplikasi lain cepat (tanda mencurigakan).",
    ],
    "DELIVERY": [
      "Bungkusan anda bermasalah. Selesaikan sekarang: [LINK]",
      "Yuran penghantaran perlu dibayar hari ini. Sila semak saluran rasmi.",
      "Status bungkusan: tindakan diperlukan dalam [TIME]. [LINK]",
      "Kami gagal menghantar. Sahkan alamat di sini: [LINK]",
      "Diminta bayaran kecil untuk lepaskan bungkusan (tanda penipuan).",
    ],
    "JOB": [
      "Kerja dari rumah, gaji tinggi, tanpa pengalaman. Sertai: [LINK]",
      "Pengambilan segera, pendapatan dijamin (amaran).",
      "Jana komisen melalui tugasan mudah. Mula: [LINK]",
      "Bayar 'yuran pendaftaran' untuk dapat kerja (tanda penipuan).",
      "Kami perlukan dokumen anda segera (tekanan masa).",
    ],
    "IMPERSONATION": [
      "Ini sokongan [ORG]. Sahkan akaun anda sekarang: [LINK]",
      "Mesej nampak rasmi tapi minta tindakan segera (amaran).",
      "Saya 'boss' anda. Perlukan pembelian segera (corak penipuan).",
      "Akaun akan disekat jika anda tak bertindak sekarang (taktik takut).",
      "Autoriti + segera + minta duit = risiko tinggi.",
    ],
    "OTHER": [
      "Anda menang hadiah. Tuntut dengan bayar yuran kecil (tanda penipuan).",
      "Pengesahan segera diperlukan. Jangan kongsi kod atau kata laluan.",
      "Jangan klik pautan asing. Gunakan SafeX Scan.",
      "Telefon anda dijangkiti. Muat turun ini sekarang (amaran palsu).",
      "Bahasa ugutan minta bayaran segera adalah mencurigakan.",
    ],
  },

  "zh": {
    "BENIGN": [
      "嗨 [NAME]，我们 [TIME] 见可以吗？",
      "请你有空时看看这个文件： [LINK]",
      "RM[AMOUNT] 的收据已生成，谢谢。",
      "你方便的时候回我电话就好，不急。",
      "提醒：你的预约时间是 [TIME]。",
    ],
    "INVESTMENT": [
      "低风险高回报、轻松赚钱（常见风险信号）。了解： [LINK]",
      "“保证盈利/稳赚不赔” 常见于骗局，请先核实。",
      "每天赚 RM[AMOUNT]，简单方法就能做到。详情： [LINK]",
      "快速提现吗？无风险？这是常见诈骗话术。",
      "邀请加入“快速收益群”，请先确认来源。",
    ],
    "ROMANCE": [
      "我很在乎你，但我需要你今天帮我一点钱（风险信号）。",
      "请保密我们的关系，并立刻帮忙（操控信号）。",
      "我现在不能视频，但请相信我。能帮 RM[AMOUNT] 吗？",
      "我在外地遇到紧急情况，需要今天帮助（常见套路）。",
      "我们快点换到别的聊天软件（可疑信号）。",
    ],
    "DELIVERY": [
      "你的包裹有问题，需要立即处理： [LINK]",
      "要求今天支付小额费用放行包裹（常见骗局）。",
      "包裹状态：需在 [TIME] 内操作。详情： [LINK]",
      "无法派送，请在此确认地址： [LINK]",
      "“不处理就退回/罚款” 属于常见施压手法。",
    ],
    "JOB": [
      "在家工作、高薪、零经验（高风险信号）。加入： [LINK]",
      "立刻录用、收入保证（常见骗局特征）。",
      "做简单任务就能赚佣金。开始： [LINK]",
      "要求先交“报名费/保证金” 很可疑。",
      "催促立刻提交资料属于施压信号。",
    ],
    "IMPERSONATION": [
      "这里是 [ORG] 客服，请立即验证： [LINK]",
      "看起来很官方但要求立刻操作（风险信号）。",
      "我是你老板，现在急需你帮忙购买（常见冒充）。",
      "不操作就停用账户（恐吓手法）。",
      "权威 + 紧急 + 要钱 = 高风险组合。",
    ],
    "OTHER": [
      "你中了大奖，先交小额手续费领取（常见骗局）。",
      "需要紧急验证，请勿分享密码或验证码。",
      "不要点陌生链接，用 SafeX Scan 先检查。",
      "“设备中毒” 要求立刻下载的提示通常是假的。",
      "带威胁的语气要求立即转账很可疑。",
    ],
  }
}

PLACEHOLDERS = {
  "NAME": ["Aiman", "Siti", "Jason", "Mei", "Arun", "Nur"],
  "TIME": ["10 minutes", "today", "now", "this evening", "2pm", "tomorrow"],
  "ORG": ["a service team", "support team", "official team"],
  "AMOUNT": ["20", "50", "100", "300", "500", "1000"],
  "LINK": ["[LINK]"],
  "QR": ["[QR]"],
}

def fill_placeholders(s: str) -> str:
    for k, vals in PLACEHOLDERS.items():
        s = s.replace(f"[{k}]", random.choice(vals))
    return s

def sample_templates(lang: str, label: str, n: int):
    base = TEMPLATES[lang][label]
    return [fill_placeholders(random.choice(base)) for _ in range(n)]


## 4) Generate dataset (balanced, EN+MS+ZH)
Change `SAMPLES_PER_CLASS_PER_LANG` to make dataset bigger/smaller.


In [ ]:
# === 4) Generate dataset ===
SAMPLES_PER_CLASS_PER_LANG = 500  # 500 * 7 classes * 3 langs = 10,500 samples total

rows = []
for lang in ["en", "ms", "zh"]:
    for label in MULTICLASS_LABELS:
        for t in sample_templates(lang, label, SAMPLES_PER_CLASS_PER_LANG):
            rows.append({
                "text": t,
                "label": label,
                "binary": to_binary_label(label),
                "lang": lang,
                "source": "synthetic_safe_templates"
            })

import pandas as pd
df = pd.DataFrame(rows).sample(frac=1.0, random_state=42).reset_index(drop=True)
df.head()


In [ ]:
# === 5) Train/test split (stratified by multiclass label) ===
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

train_df.to_csv("train_multiclass.csv", index=False)
test_df.to_csv("test_multiclass.csv", index=False)

# Binary CSVs: label column must be named 'label'
train_df[["text","binary","lang","source"]].rename(columns={"binary":"label"}).to_csv("train_binary.csv", index=False)
test_df[["text","binary","lang","source"]].rename(columns={"binary":"label"}).to_csv("test_binary.csv", index=False)

print("Saved:", "train_multiclass.csv", "test_multiclass.csv", "train_binary.csv", "test_binary.csv")


## 6) Train multi‑class model (average_word_vec)


In [ ]:
spec = model_spec.get('average_word_vec')

train_data = DataLoader.from_csv(
    filename='train_multiclass.csv',
    text_column='text',
    label_column='label',
    model_spec=spec,
    is_training=True
)
test_data = DataLoader.from_csv(
    filename='test_multiclass.csv',
    text_column='text',
    label_column='label',
    model_spec=spec,
    is_training=False
)

model_mc = text_classifier.create(train_data, model_spec=spec, epochs=10)
loss, acc = model_mc.evaluate(test_data)
print("Multi-class loss:", loss, "accuracy:", acc)


In [ ]:
EXPORT_DIR_MC = "safex_triage_multiclass"
model_mc.export(export_dir=EXPORT_DIR_MC)

tflite_path_mc = os.path.join(EXPORT_DIR_MC, "model.tflite")
tflite_acc_mc = model_mc.evaluate_tflite(tflite_path_mc, test_data)

print("Exported multi-class TFLite accuracy:", tflite_acc_mc)
print("Multi-class TFLite saved at:", tflite_path_mc)


## 7) Train binary model (SCAM vs BENIGN)


In [ ]:
spec_bin = model_spec.get('average_word_vec')

train_bin = DataLoader.from_csv(
    filename='train_binary.csv',
    text_column='text',
    label_column='label',
    model_spec=spec_bin,
    is_training=True
)
test_bin = DataLoader.from_csv(
    filename='test_binary.csv',
    text_column='text',
    label_column='label',
    model_spec=spec_bin,
    is_training=False
)

model_bin = text_classifier.create(train_bin, model_spec=spec_bin, epochs=8)
loss_b, acc_b = model_bin.evaluate(test_bin)
print("Binary loss:", loss_b, "accuracy:", acc_b)

EXPORT_DIR_BIN = "safex_triage_binary"
model_bin.export(export_dir=EXPORT_DIR_BIN)

tflite_path_bin = os.path.join(EXPORT_DIR_BIN, "model.tflite")
tflite_acc_bin = model_bin.evaluate_tflite(tflite_path_bin, test_bin)

print("Exported binary TFLite accuracy:", tflite_acc_bin)
print("Binary TFLite saved at:", tflite_path_bin)


## 8) Download
In Colab, click the **folder icon** (Files) on the left sidebar.
- Open `safex_triage_multiclass/` → right-click `model.tflite` → **Download**
- Open `safex_triage_binary/` → right-click `model.tflite` → **Download**

Then copy the model into your Android project:
`app/src/main/assets/`
